In [30]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Group Assignment
### Team Number: 05
### Team Member Names: Piero Camposeo, Sathun Suthakaran, Ishaan Bansal
### Team Strategy Chosen: SAFE

*Abstract and explanation go here*

In [31]:
#Read in csv file of tickers
csv_file = "Tickers.csv"

# Make values into a list
tickers = pd.read_csv(csv_file)
ticker_list = tickers.iloc[:, 0].values.tolist()
ticker_list.insert(0, tickers.columns[0])

In [32]:
#Consumes desired amount of trading days within a month, a dataframe of days, a dstaframe of months, and a list of months that cooresponds to the months
#    on the dataframe, outputs the monthly dataframe with only months that have 'daycount' amount of trading days for that stock
#Note: Function will cover up to the last date. For example, the function will not produce October for a dataframe which ends on October 1st.
#   In our code, we simply passed a days dataframe that ends one month after the months dataframe does (*)
def df_with_valid_months(daycount, df_days, df_months, months):
    for month in range (len(months)):
        if len(df_days.filter(like = months[month], axis = 0)) < daycount:
            df_months.drop(df_months.index[month], axis = 0, inplace = True) #Drop any months with not enough trading days
            
    return(df_months)

#Consumes historical data for a stock, outputs average volume
def average_volume(df_hist):
    volume = df_hist['Volume'].dropna().values.tolist()
    vol_av = np.mean(volume)

    return(vol_av)


In [33]:
#Consumes a list of tickers, outputs tickers that:
#   a) Are US listed
#   b) Have an average monthly volume of 200 000 shares
def filter_stocks(ticker_list):
    #dates for volume calculation, end date for valid months function (see (*))
    vol_date_s = '2022-01-01'
    vol_date_e = '2022-10-02'
    endDForValidMonthsFunct = str((pd.to_datetime(vol_date_e) + relativedelta(months = 1)).strftime("%Y-%m-%d"))

    #As outlined in assignment information
    minimumMonthlyTradingDays = 20
    minimum_volume = 200000

    #Here we will put valid stocks that fit our criteria
    newlist = []
    
    #for df_with_valid_months function, created a series of months from vol_date_s to vol_date_e
    months = pd.date_range(vol_date_s, vol_date_e, freq='MS').strftime("%Y-%m").tolist()

    #Checks for criteria needed to be a valid ticker
    for ticker in ticker_list:
        tick = yf.Ticker(ticker)
        tick_hist = tick.history(start = vol_date_s, end  = vol_date_e, interval = '1mo').dropna()
        
        try:
            #Custom dataframe for valid months check (daily, ending one month after monthly dataframe)
            tickHistForValidMonthsCheck = tick.history(start = vol_date_s, end  = endDForValidMonthsFunct, interval = '1d').dropna()
            #Dataframe with valid months only
            tickHistWithValidMonths = df_with_valid_months(minimumMonthlyTradingDays, tickHistForValidMonthsCheck, tick_hist, months)

            monthly_average_volume = 0 #Initialize this to 0 to prevent errors
            monthly_average_volume = average_volume(tickHistWithValidMonths)

            currency = tick.info['financialCurrency']
        
            #Check for criteria
            if (currency == 'USD' and monthly_average_volume >= minimum_volume):
                    newlist.append(ticker)
        #Function will throw an error if the stock isn't valid due to the attempt to call on its .info
        except:
            print(f"Error retrieving {ticker}.")

    return(newlist)
    
#Final list of valid tickers
ticker_list = filter_stocks(ticker_list)

Error retrieving AAPL.


In [ ]:
#Sathun

In [ ]:
#Ishaan

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Piero Camposeo, Sathun Suthakaran, Ishaan Bansal